In [2]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [7]:
en(['.'])
d([[3, 5, 1]])

['.']

In [15]:
fake_B = torch.zeros((3,4,5))
fake_B[0][0][1] = 1
fake_B[0][1][0] = 1
fake_B[0][2][3] = 1
fake_B

tensor([[[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

In [16]:

a  = (fake_B[:, :,0] != 1).long()

In [9]:
model = torch.load('./model/D_B.pt').eval()

In [12]:
#real sentences
x = [[3, 5, 1, 
0,0,0,0,0,0]]

             #delte 0 give huge diff
       
x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret =  model.classifier(distr)#(bs,1)
print(ret)

torch.Size([1, 9])
		---------embedding
torch.Size([1, 9, 512])
tensor([[[ 11.1214,   7.9124,  14.1215,  ...,   9.6461,  -7.8171,  -3.5773],
         [ -4.2265,   7.2603, -14.4783,  ...,  12.9361,  -2.6143,  -3.0168],
         [ 12.5392,   8.0912, -11.4242,  ...,   7.7111,  -7.1866,   0.8568],
         ...,
         [ -1.8653,   0.0877,  -6.9863,  ...,  -0.3819,   2.6015,  -2.7637],
         [ -1.8653,   0.0877,  -6.9863,  ...,  -0.3819,   2.6015,  -2.7637],
         [ -1.8653,   0.0877,  -6.9863,  ...,  -0.3819,   2.6015,  -2.7637]]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)
		---------encode
torch.Size([1, 9, 512])
tensor([[[ 0.0265, -0.0650, -0.1443,  ...,  0.1849, -0.0047, -0.0293],
         [-0.0760, -0.1067, -0.0037,  ..., -0.0455, -0.0526, -0.0423],
         [ 0.0402,  0.0716, -0.1324,  ...,  0.0232,  0.0016,  0.0740],
         ...,
         [ 0.0589, -0.0985,  0.0467,  ...,  0.0047,  0.0442,  0.0597],
         [ 0.0548, -0.0999,  0.0448,  ...,  0.0122,  0.0447,  0.

In [13]:
#real sentences
x = [[10180,   351, 19810,     7,    24,     6,    38,     8,  8160,  8675,
            13,     8,  1611, 15375,     6,    62,   225,     6,    11,  5071,
           398,     6,  2173,   981, 12361,   485,    11, 13567,    16,     8,
          1028,  5808,     7,  1194,    11,  6572,    53,    13,    48,   540,
             6, 10321,    69, 12123,     7,    11,   811,     7,  9098,    12,
             3,  9582,   125,    33,   801,    38,    96, 12097,    21,   540,
           121, 15600,    16,     8,  5334,    18, 14259,   433,     5,     1,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]

             #delte 0 give huge diff
       
x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret =  model.classifier(distr)#(bs,1)
print(ret)

torch.Size([1, 90])
		---------embedding
torch.Size([1, 90, 512])
tensor([[[  5.9575,  -7.8626,  14.3691,  ..., -21.0999,   3.9790,  12.4444],
         [ 23.7227,  -2.7165, -15.8925,  ...,  25.1313,  -6.2685,   2.0443],
         [  5.2379, -38.2814,  46.2259,  ...,  -6.0190, -31.0495,  -2.6075],
         ...,
         [ -1.8653,   0.0877,  -6.9863,  ...,  -0.3819,   2.6015,  -2.7637],
         [ -1.8653,   0.0877,  -6.9863,  ...,  -0.3819,   2.6015,  -2.7637],
         [ -1.8653,   0.0877,  -6.9863,  ...,  -0.3819,   2.6015,  -2.7637]]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)
		---------encode
torch.Size([1, 90, 512])
tensor([[[ 0.3019, -0.0240,  0.1268,  ..., -0.4100, -0.0095,  0.0712],
         [ 0.3483, -0.0669,  0.2359,  ..., -0.2571,  0.0019,  0.0086],
         [ 0.0245,  0.1051,  0.3846,  ..., -0.0195,  0.0081,  0.0784],
         ...,
         [ 0.1691, -0.0201,  0.1806,  ..., -0.1905,  0.0086,  0.1166],
         [ 0.1677, -0.0203,  0.1807,  ..., -0.1902,  0.0084, 

In [45]:

# x_emb = model.embedding(x_)#
# print('x_emb.shape',x_emb.shape)
# distr = model.encoder(inputs_embeds=x_emb).last_hidden_state#(bs,sentence length,512)
# print('distr.shape',distr.shape)
# x_attn= x_attn.unsqueeze(-1)
# print('x_attn.shape',x_attn.shape)
# distr = torch.mul(distr,x_attn)#previously

# print('mutil',torch.sum(distr,1))
# print('count',torch.sum(x_attn,1))
# distr = torch.sum(distr,1)/torch.sum(x_attn,1)#(bs,512)
# print(torch.mean(distr,-1))
# print('mea',distr.shape)

# ret =  model.classifier(distr)#(bs,1)
# print(ret)
# ret = model.relu(ret)#(bs,1)
# print(ret)



a_pred_dis = model(x_,x_attn)
a_pred_dis

tensor([[0.9008]], device='cuda:0', grad_fn=<ReluBackward0>)

,attention_mask=x_attn is essential for the encoder inpuit


## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier